In [ ]:
import subprocess
import pathlib
import os, sys

# load in the workspace dir
current_dir = pathlib.Path.cwd()
WORKSPACE_DIR = None
while current_dir != current_dir.parent:
    if (current_dir / "neutralb1").is_dir():
        WORKSPACE_DIR = str(current_dir / "neutralb1")        
    current_dir = current_dir.parent
if WORKSPACE_DIR is None:
    raise FileNotFoundError("Could not find 'neutralb1' directory in the path.")

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

d940728e426b01b59832a3bf43f791e3e2a51927


**Repository Version** 
This notebook was run at commit:
`FILL WHEN DONE`

# Verifying the Projection of Moments with Signal MC
As found previously, the projected moments have some missing factors causing them not to match expectations. Since then, two major updates have occured:
1. I've developed a direct fit to data using moments as AmpTools parameters, which should now provide a set of "true" values we can compare the projected moments to.
   <br>a. Has issues in extracting moments with $>1\%$ contribution, but this should be enough to track down factors
2. The old python projection script has now been replaced by a c++ version, that also includes the necessary normalization integrals
   a. The script may likely be updated over time, so check the commit hash for what version to use.


This study will proceed as follows:
1. Generate Signal Monte Carlo (MC) according to a pseudo-realistic set of waves (no acceptance effects i.e. *thrown*)
   <br>a. 35% polarization and in the PARA_0 orientation
2. Fit MC with same waveset, and obtain a fit result that should match the generated wave values
3. Project moments from the fit result to obtain a projected moment-set $H_{\text{proj}}$
4. Fit MC with the same number of moments, and obtain a fitted moment-set $H_{\text{fit}}$
5. Compare the fit and projected sets to investigate the missing factors.

## Setup

In [ ]:
# load libraries
import pandas as pd
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import src.utils.utils as utils
import src.analysis.pwa_tools as pwa_tools

utils.load_environment()
plt.style.use(f"{WORKSPACE_DIR}/config/pwa_plotter.mplstyle")

# get the cwd
CWD = pathlib.Path.cwd()

bash: line 1: /w/halld-scshelf2101/kscheuer/neutralb1/setup_gluex.sh: No such file or directory


### Generate MC and fit with amplitudes
We'll use the same cfg file to generate and fit the amplitude-based Monte Carlo with. This will be done in a single bin of mass at:
* $0.1 < -t < 0.2$
* $8.2 < E_\gamma < 8.8$
* $1.20 < M_{\omega\pi^0} < 1.22$

In [10]:
with open(f"{CWD}/amplitudes.cfg", "r") as f:
    for i in range(20):
        print(f.readline(), end="")

# This cfg file is designed to generate and fit waves in a single mass bin
# to confirm the projection of moments is working correctly. No acceptance effects
# will be applied, and the phasespace will contain the same TEM cuts as the signal
#   -t: 0.1 - 0.2
#    M: 1.2 - 1.22
#    E: 8.2 - 8.8
#
# The waveset is pseudo-realistic, and comes from the previous truth-initialized fits,
# but has already applied the Breit-Wigner and scale parameter to the complex values


# generator-level configuration parameters
define vector 0.782 0.008
define beamconfig beam.cfg

# fit setup
fit omegapi
normintfile omegapi omegapi.ni
reaction omegapi Beam Proton Pi01 Pi02 Pi+ Pi-
sum omegapi ImagNegSign RealNegSign RealPosSign ImagPosSign Bkgd
